# Assigment 4| Interactive Visualizations in Python
## By Vyanna Hill

### Intro to Dash

In the previous module, I built a shiny app in R for users to interact with the data and select visualization for analysis. This week we will enact the same principles but in Python' Dash app. The Dash library can create apps where users can interact with the program to display the chosen app. 

For this week's assignment, I used Dash's documentation to help create my visualizer app and the dash jupyter documentation to enable the app inside the [notebook](https://github.com/plotly/jupyter-dash). I will go through the steps of building the app below.

In [1]:
#Loading in the libraries
import pandas as pd
import numpy as np
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

### Part 2| Graphing plotly

From my experience in module two, My device cannot handle large data with plotly. Following the module's sample notebook, it will be wise to create a query for the needed data below. I created two versions of the data selected with SoQL. This way, the query can bypass the api limitations on data and the data grouping is eaiser on loading.

The first query will contain the borough of the tree, it's specices, and the count of the health status. The second query will all of the first query's variables and the steward varible. The two query have all the necessary information asked for in assignment's questions and mindful of the size of the data. Next, we will move onto the appropirate visualization.

For the first graph, we can show the proportion of trees by its health status with a bar graph. It's the most fitting visualization as we can group by borough and filter the plot by health. The second graph,it was the most confusing one to develop. First, I though of a correlation map but my variables made the plot diffcult to view/ understand. For simipilicty, we can also go with a bar graph.

In [202]:
#We use jupyterdash to create our app inside the notebook instead of calling a seperate file like the video example in the weekly folder
#dbc is use for style themeing the app
app = JupyterDash(__name__,external_stylesheets=[dbc.themes.SIMPLEX])

#The front end portion resides below, all parts of our app are gather under the layout and grouped within the div statemet
app.layout=html.Div([
    #Using the html library, we can add in html elements for the front end of the app
    html.H1("Dash| Visualization interactivity in Python"),
    html.H2("By Vyanna Hill"),\
    
    #dcc added in the option for componenments for our app
    dcc.Dropdown(['Tree Health in NYC','Steward Activity vs Health'],'Tree Health in NYC',id="drop_in"),

    #Resulting graph from user's input
    dcc.Graph(id="output_plot")
    
])

#The backend of the app, the app callback assigns variables to the inputs from the user and its outputs in the frontend
@app.callback(
    Output("output_plot","figure"),
    Input("drop_in","value")
)

#funcitons below give the interactivity of the app
#User will envoke the function once a input value is selectd
def update_graph(value):
    if(value=="Tree Health in NYC"):
        #I created a query that selects borough name, tree species, its health grouped by count. Borrowed the sample url builder to combine it all
        soql_url = ('https://data.cityofnewyork.us/resource/uvpi-gqnh.json?' +\
        '$select=boroname,spc_common,health,count(tree_id)'+\
        '&$group=spc_common,boroname,health&$limit=5000').replace(' ', '%20')
        data = pd.read_json(soql_url)
        data.dropna(inplace=True)
        
        #creating a bar graph with plotly, create subplots for each borough as specifiy 
        fig=px.bar(soql_trees,x="count_tree_id",y="spc_common",color="health",facet_col="boroname",orientation='h')
        fig.for_each_xaxis(lambda x: x.update(title = ''))
        fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
        fig.update_layout(title="Proportion of NYC Trees' Health by Species",xaxis_title="Tree Count",yaxis_title="Tree Species",legend_title="Health")
        return fig
    else:
        #Following the steps above, lets create of game plan in nycopen visualize tool and export the api url
        soql_url=('https://data.cityofnewyork.us/resource/uvpi-gqnh.json?' +\
        '$select=boroname,spc_common,steward,health,count(tree_id)'+\
        '&$group=spc_common,boroname,steward,health&$limit=5000').replace(' ', '%20')
        data=pd.read_json(soql_url)
        data.dropna(inplace=True)

        #creating a bar graph with plotly
        fig=px.bar(soql_trees,x="count_tree_id",y="spc_common",color="health",facet_col="steward", facet_row="boroname",orientation='h')

        #each annotation line from plotly's bar documentation,remove "=" from the annotation. Then, remove the duplicate x/y-axis titles for formatting 
        fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
        fig.for_each_yaxis(lambda y: y.update(title = ''))
        fig.for_each_xaxis(lambda x: x.update(title = ''))
        fig.update_layout(title="Impact of Stewards on NYC Trees",xaxis_title="Tree Count",yaxis_title="Tree Species",legend_title="Health")
        return fig


if __name__ == '__main__':
    app.run_server(mode="inline")